# MODELE XGBOOST CALIBRE + SOUMISSION

## CHARGEMENT DES LIBRAIRIES

In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [ ]:
#!pip install category_encoders > /dev/null 2>&1
#!pip install xgboost > /dev/null 2>&1

In [1]:
import pandas as pd
import numpy as np
import panel as pn

from sklearn.compose import make_column_selector as selector
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from category_encoders import TargetEncoder, MEstimateEncoder, CatBoostEncoder
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

## PREPROCESSING

In [18]:
#data = pd.read_csv('/content/gdrive/My Drive/Defi-IA/new_result_requetes.csv')
data = pd.read_csv('new_result_requetes.csv')

In [19]:
#On affecte le bon type aux variables qualitatives
data["avatar_id"]=pd.Categorical(data["avatar_id"],ordered=False)
data["city"]=pd.Categorical(data["city"],ordered=False)
data["language"]=pd.Categorical(data["language"],ordered=False)
data["mobile"]=pd.Categorical(data["mobile"],ordered=False)
data["hotel_id"]=pd.Categorical(data["hotel_id"],ordered=False)
data["group"]=pd.Categorical(data["group"],ordered=False)
data["brand"]=pd.Categorical(data["brand"],ordered=False)
data["parking"]=pd.Categorical(data["parking"],ordered=False)
data["pool"]=pd.Categorical(data["pool"],ordered=False)
data["children_policy"]=pd.Categorical(data["children_policy"],ordered=False)

data["price"]=pd.DataFrame(data["price"], dtype=float)

In [20]:
#X = data[["nb_requete","date","stock","city","language","mobile","hotel_id","group","brand","parking","pool","children_policy"]]
X = data[["date","stock","city","language","mobile","hotel_id","group","brand","parking","pool","children_policy"]]
#X = data[["date","stock","city","language","hotel_id","group","brand"]]

In [21]:
Y = data[["price"]]

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [23]:
#y_train = np.ravel(y_train)
#y_test = np.ravel(y_test)

## TARGET SMOOTHING

In [24]:
def calc_smooth_mean(df, by, on, m):
    # Compute the global mean
    mean = df[on].mean()

    # Compute the number of values and the mean of each group
    agg = df.groupby(by)[on].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']

    # Compute the "smoothed" means
    smooth = (counts * means + m * mean) / (counts + m)

    # Replace each value by the according smoothed mean
    return df[by].map(smooth)

In [25]:
dataT = X_train.join(y_train, on=None, how='right', lsuffix='', rsuffix='', sort=False)
dataT

,date,stock,city,language,mobile,hotel_id,group,brand,parking,pool,children_policy,price
180999,41,92,vilnius,luxembourgish,0,648,Independant,Independant,1,0,0,57.0
375785,10,4,rome,bulgarian,0,953,Boss Western,Boss Western,0,0,0,123.0
292699,14,15,vienna,finnish,0,26,Accar Hotels,Marcure,1,0,0,199.0
379417,5,5,madrid,italian,1,768,Accar Hotels,Safitel,0,1,0,364.0
14624,21,25,vilnius,polish,1,408,Morriott International,CourtYord,1,0,0,108.0
...,...,...,...,...,...,...,...,...,...,...,...,...
259178,0,0,copenhagen,slovakian,0,481,Independant,Independant,1,0,0,93.0
365838,31,78,vienna,irish,0,219,Morriott International,Morriot,0,1,0,171.0
131932,20,7,vienna,croatian,0,919,Yin Yang,8 Premium,0,0,0,76.0
146867,17,0,rome,swedish,0,636,Independant,Independant,0,0,0,102.0


In [26]:
dataT['city'] = calc_smooth_mean(data, by='city', on='price', m=300)
dataT['language'] = calc_smooth_mean(data, by='language', on='price', m=300)
dataT['mobile'] = calc_smooth_mean(data, by='mobile', on='price', m=300)
dataT['hotel_id'] = calc_smooth_mean(data, by='hotel_id', on='price', m=300)
dataT['group'] = calc_smooth_mean(data, by='group', on='price', m=300)
dataT['brand'] = calc_smooth_mean(data, by='brand', on='price', m=300)
dataT['parking'] = calc_smooth_mean(data, by='parking', on='price', m=300)
dataT['pool'] = calc_smooth_mean(data, by='pool', on='price', m=300)
dataT['children_policy'] = calc_smooth_mean(data, by='children_policy', on='price', m=300)

In [27]:
dataT

,date,stock,city,language,mobile,hotel_id,group,brand,parking,pool,children_policy,price
180999,41,92,112.579633,156.952975,159.141817,103.567833,96.370483,96.370483,159.852509,141.311186,156.041313,57.0
375785,10,4,149.131076,154.134707,159.141817,137.684990,134.879991,123.672497,157.759659,141.311186,156.041313,123.0
292699,14,15,156.135104,164.186038,159.141817,168.085851,188.957295,199.226930,159.852509,141.311186,156.041313,199.0
379417,5,5,140.337542,161.028454,158.203332,240.184242,188.957295,294.467693,157.759659,243.636854,156.041313,364.0
14624,21,25,112.579633,158.753352,158.203332,138.954049,225.321703,158.995270,159.852509,141.311186,156.041313,108.0
...,...,...,...,...,...,...,...,...,...,...,...,...
259178,0,0,199.489805,155.773562,159.141817,123.115929,96.370483,96.370483,159.852509,141.311186,156.041313,93.0
365838,31,78,156.135104,161.089841,159.141817,189.519899,225.321703,198.033458,157.759659,243.636854,156.041313,171.0
131932,20,7,156.135104,161.539876,159.141817,106.595150,150.950896,77.522402,157.759659,141.311186,156.041313,76.0
146867,17,0,149.131076,162.023432,159.141817,120.657963,96.370483,96.370483,157.759659,141.311186,156.041313,102.0


In [28]:
XT_train=dataT.drop(columns=["price"])
XT_train

,date,stock,city,language,mobile,hotel_id,group,brand,parking,pool,children_policy
180999,41,92,112.579633,156.952975,159.141817,103.567833,96.370483,96.370483,159.852509,141.311186,156.041313
375785,10,4,149.131076,154.134707,159.141817,137.684990,134.879991,123.672497,157.759659,141.311186,156.041313
292699,14,15,156.135104,164.186038,159.141817,168.085851,188.957295,199.226930,159.852509,141.311186,156.041313
379417,5,5,140.337542,161.028454,158.203332,240.184242,188.957295,294.467693,157.759659,243.636854,156.041313
14624,21,25,112.579633,158.753352,158.203332,138.954049,225.321703,158.995270,159.852509,141.311186,156.041313
...,...,...,...,...,...,...,...,...,...,...,...
259178,0,0,199.489805,155.773562,159.141817,123.115929,96.370483,96.370483,159.852509,141.311186,156.041313
365838,31,78,156.135104,161.089841,159.141817,189.519899,225.321703,198.033458,157.759659,243.636854,156.041313
131932,20,7,156.135104,161.539876,159.141817,106.595150,150.950896,77.522402,157.759659,141.311186,156.041313
146867,17,0,149.131076,162.023432,159.141817,120.657963,96.370483,96.370483,157.759659,141.311186,156.041313


In [36]:
listlanguage = ['austrian', 'belgian', 'bulgarian', 'croatian', 'cypriot', 'czech', 'danish', 'dutch', 'estonian', 'finnish',
            'french', 'german', 'greek', 'hungarian', 'irish', 'italian', 'latvian', 'lithuanian', 'luxembourgish', 
            'maltese', 'polish', 'portuguese', 'romanian', 'slovakian', 'slovene', 'spanish','swedish']
listcity = ['amsterdam', 'copenhagen', 'madrid', 'paris', 'rome', 'sofia', 'valletta', 'vienna', 'vilnius']
listmobile = ['0','1']

listhotel_id = np.arange(0,999,1)
listgroup = ['Independant','Accar Hotels','Yin Yang','Boss Western','Morriott International','Chillton Worldwide']
listbrand = ['Independant','Boss Western','J.Halliday Inn','Royal Lotus','Safitel','Corlton','Ardisson','Marcure',
         '8 Premium','Morriot','Tripletree','Quadrupletree','Ibas','Chill Garden Inn','Navatel','CourtYord']
listparking = ['0','1']
listpool = ['0','1']
listchildren_policy = ['0','1','2']

#### IDEE :
pour chq liste, créer un vecteur(dataframe?) de même taille dans lequel on va mettre la valeur de la moyenne qui correspond à la modalité et qui a été calculée par la fonction target.

exemple : pour listlanguage on crée le tableau targetlangage. la 1re case sera la moyenne qui correspond à l'austrian. pour obtenir cette moyenne, il faut parcourir la colonne du X_train'language' et dès qu'on tombe sur 'austrian', on prend la valeur du XT_train'language' qui correspond : ce sera la même colonne et la même ligne car XT_train est une transformation du X_train. une fois qu'on est tombé sur un 'austrian', on récupère la moyenne et on arrête de parcourir la colonne. On refait pareil pour chq language.

du coup une fois qu'on a ce vecteur(dataframe?), on a l'association des modalités et des moyennes. On peut alors prendre notre X_test, le copier en XT_test et remplacer les colonnes catégorielles par les moyennes en regardant dans le tableau des moyennes à quoi ça correspond.

exemple : pour XT_test colonne language, si on tombe sur 'austrian', on regarde dans le tableau et on prend la moyenne qui correspond.

Et il faudra aussi faire cette transformation pour le fichier test du Defi.

In [37]:
targetlanguage = np.zeros(len(listlanguage))
targetlanguage

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
x = np.ravel(X_train[['city']])
xT = np.ravel(XT_train[['city']])
for i in range(len(x)):
#???

## APPRENTISSAGE DE MODELE

In [ ]:
y_train = np.ravel(y_train)
y_test = np.ravel(y_test)

In [ ]:
pip = Pipeline(steps=[("Standard_scaler", StandardScaler()),
                      ("Boosting", XGBRegressor(n_estimators=3000,max_depth=10)),
                      ]
               )

In [ ]:
pip

### A EXECUTER SI PAS DE GRIDSEARCH

In [ ]:
pip.fit(X_train, y_train)

In [ ]:
print(f"pip score: {pip.score(X_test, y_test):.2f}")

In [ ]:
### CROSS VAL SCORE?

### A EXECUTER SI GRIDSEARCH

In [ ]:
# param_grid pour le GradientBoosting

# n_estimators + grand? 100 D
# max_features?
# max_depth? 3 D

#Best params:
#{'Boosting__n_estimators': ..., 'Boosting__max_depth': ...}

#param_grid = {
#    "Boosting__n_estimators":[100,300,500],
#    #"Boosting__max_features":[1.,0.5,0.2],
#    "Boosting__max_depth":[7,10,13],
#}

In [ ]:
# param_grid pour le HistGradientBoosting

# learning_rate? 0.1 D
# max_iter? 100 D
# max_leaf_nodes? 31 D
# max_depth? no D

#Best params:
#{'Boosting__max_depth': 10, 'Boosting__max_iter': 3000} ???
#{'Boosting__max_depth': 13, 'Boosting__max_iter': 5000} ???

param_grid = {
#    #"Boosting__learning_rate": (0.01, 0.1, 1, 10),
    "Boosting__max_iter":[1000, 3000, 5000],
#    #"Boosting__max_leaf_nodes": [30, 50, 70],
    "Boosting__max_depth": [7, 10, 13],
    }

In [ ]:
# cv? 5 D
grid_search = GridSearchCV(pip, param_grid, cv=5, n_jobs=-1)
grid_search

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
print("Best params:")
print(grid_search.best_params_)

In [ ]:
print(f"Internal CV score: {grid_search.best_score_:.3f}")

In [ ]:
print(f"GridSearch score: {grid_search.score(X_test, y_test):.3f}")

## PLOT + MSE (pip/grid_search à choisir)

In [ ]:
import matplotlib.pyplot as plt
y_hat = pip.predict(X_test)
#y_hat = grid_search.predict(X_test)

plt.plot(y_hat,y_test,"o")
plt.xlabel(u"prix prédit")
plt.ylabel(u"prix observé")
plt.show()

In [ ]:
plt.plot(y_hat,y_test-y_hat,"o")
plt.xlabel(u"prix prédits")
plt.ylabel(u"résidus")
plt.hlines(0,0,500)
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
print("RMSE=",mean_squared_error(y_test,y_hat, squared=False))
#RMSE = 6.14 pour Cat-Hist 10/3000
#RMSE = 6.05 pour Cat-Hist 15/5000
#RMSE = 5.89 pour Cat-Hist 20/10000
#RMSE = 6.01 pour Cat-GB 10/300
#RMSE = 5.43 pour Cat-GB 10/1000
#RMSE = 1.35 pour M-GB 10/1000
#RMSE = 1.35 pour T-GB 10/1000
#RMSE = 1.17 pour T-GB 10/2000
#RMSE = 1.16 pour T-GB 10/3000

#12/12
#RMSE = 1.17 pour T-GB 10/2000
#RMSE = 1.49 pour T-GB-7 10/2000      
#RMSE = 15.26 pour T-GB-6 10/2000 !!!

#RMSE = 1.15 pour T-XGB-11 10/2000
#RMSE = 1.45 pour T-XGB-7 10/2000
#RMSE = 14.98 pour T-XGB-6 10/2000 !!!

#RMSE = 2.01 pour O-XGB-12 10/3000
#RMSE = 3.15 pour O-XGB-11 10/2000
#RMSE = 2.09 pour O-XGB-11 10/3000
#RMSE = 1.47 pour O-XGB-11 10/5000
#RMSE = 3.43 pour O-XGB-7 10/2000

In [ ]:
from sklearn.metrics import r2_score
print("R2=",r2_score(y_test,y_hat))

### POUR ENREGISTRER LE MODELE (pip/grid_search à choisir)

In [ ]:
#from joblib import dump, load
#dump(pip, 'XGB_model_saved.joblib')
#pip_loaded = load('XGB_model_saved.joblib')

## TEST DU MODELE SUR LE FICHIER DEFI

In [ ]:
test = pd.read_csv('/content/gdrive/My Drive/Defi-IA/test_set3.csv')
#test = pd.read_csv('./defi-ia-2023/test_set3.csv')

In [ ]:
#On affecte le bon type aux variables qualitatives
test["avatar_id"]=pd.Categorical(test["avatar_id"],ordered=False)
test["city"]=pd.Categorical(test["city"],ordered=False)
test["language"]=pd.Categorical(test["language"],ordered=False)
test["mobile"]=pd.Categorical(test["mobile"],ordered=False)
test["hotel_id"]=pd.Categorical(test["hotel_id"],ordered=False)
test["group"]=pd.Categorical(test["group"],ordered=False)
test["brand"]=pd.Categorical(test["brand"],ordered=False)
test["parking"]=pd.Categorical(test["parking"],ordered=False)
test["pool"]=pd.Categorical(test["pool"],ordered=False)
test["children_policy"]=pd.Categorical(test["children_policy"],ordered=False)

In [ ]:
T = test[["nb_requete","date","stock","city","language","mobile","hotel_id","group","brand","parking","pool","children_policy"]]
#T = test[["date","stock","city","language","mobile","hotel_id","group","brand","parking","pool","children_policy"]]
#T = test[["date","stock","city","language","hotel_id","group","brand"]]

#### PREDICTION (pip/grid_search à choisir)

In [ ]:
Yprev=pip.predict(T)
#Yprev=grid_search.predict(T)

In [ ]:
Yprev0=np.around(Yprev, decimals=1)

In [ ]:
Yindice=pd.DataFrame(np.arange(0,len(Yprev)),columns = ['index'])

In [ ]:
Yprice=pd.DataFrame(Yprev0,columns = ['price'])
#Yprice=pd.DataFrame(Yprev,columns = ['price'])
Ysub=Yindice.join(Yprice, on=None, how='right', lsuffix='', rsuffix='', sort=False)

In [ ]:
Ysub.to_csv('/content/gdrive/My Drive/Defi-IA/sample_submission77-O-XGB11-10-3000.csv', index= False)
#Ysub.to_csv('./submission/sample_submissionXGB.csv', index= False)

#INTERPRETABILITE
#### pip/grid_search à choisir

In [ ]:
feature_names = X_test.columns
feature_names

In [ ]:
model_name = 'Pip'
model = pip
#model_name = 'Grid Search'
#model = grid_search

## Permutation Importance

In [ ]:
!pip install eli5 > /dev/null 2>&1

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure()
permutation_impor = PermutationImportance(model, random_state=0).fit(X_test, y_test)

In [ ]:
features_importance

In [ ]:
plt.figure()
ax = sns.barplot(x="Importance", y="Feature_name", data=features_importance)
plt.title(model_name)

## SHAP

In [ ]:
!pip install shap > /dev/null 2>&1

In [ ]:
import shap
shap.initjs() #needed to plot results directly on the notebook

idx = 1 # index of the instance we want to explain

explainer = shap.KernelExplainer(model.predict, X_test)

In [ ]:
shap_values = explainer.shap_values(X_test.iloc[0,:])
shap.force_plot(explainer.expected_value, shap_values, X_test.iloc[0,:])
#shap_values = explainer.shap_values(X_test.iloc[0:10,:]) #To speed up we just compute the shap values for 100 exemples
#shap.summary_plot(shap_values, X_test.iloc[0:10,:])